In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as wtoken
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import (Activation, Dropout, Dense, Embedding, Flatten, BatchNormalization, GlobalMaxPooling1D, Lambda, concatenate, LSTM, Concatenate)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os, string, gc, re

from tqdm import tqdm


import tldextract as tld


from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

# In addition to Tensorflow, pandas and sklearn, this notebook requires the following NLP packages.
- conda install -c anaconda nltk
- conda install -c conda-forge tldextract
- conda install -c conda-forge wordcloud

In [2]:
# A quick verification to see if the gpu drivers are ready to serve traning DL
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
Num GPUs Available:  1


In [3]:
features = ['keyword', 'location', 'text']

In [4]:
path = '/media/moh/abunayla/Disaster_Tweets/Notebooks/data/'

In [5]:
train = pd.read_csv(path + 'train.csv', usecols = features)

test = pd.read_csv(path + 'test.csv', usecols = features)

samsub = pd.read_csv(path +'sample_submission.csv')

Y =  pd.read_csv(path +'train.csv', usecols = ['target'])

In [6]:
train.head()

,keyword,location,text
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,NaN,NaN,All residents asked to 'shelter in place' are ...
3,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...


In [9]:
train.isna().sum()

keyword       61
location    2533
text           0
dtype: int64

In [10]:
test.isna().sum()

keyword       26
location    1105
text           0
dtype: int64

In [12]:
samsub.isna().sum()

id        0
target    0
dtype: int64

In [16]:
train.location.value_counts()

USA                     104
New York                 71
United States            50
London                   45
Canada                   29
                       ... 
Queen Creek AZ            1
oklahoma                  1
Essex/Brighton            1
-6.152261,106.775995      1
cleveland, oh             1
Name: location, Length: 3341, dtype: int64

In [19]:
Y.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [20]:
4342 - 3271

1071